In [2]:
import torch as torch
import numpy as np
from scipy.sparse import load_npz
import timeit

In [3]:
device = torch.device("cuda")
print(device)

cuda


In [33]:
torch.cuda.init()

In [4]:
D2 = load_npz("laplacian_D2_500_sw1pers.npz")

In [5]:
x = np.random.uniform(size=D2.shape[1])

In [6]:
timeit.timeit(lambda: D2 @ x, number=50)
# 5.859196637058631 = cloud cpu 
# 6.5684758230054285 = host cpu 

5.914250231347978

In [7]:
values = D2.data
indices = np.vstack((D2.row, D2.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = D2.shape
D2_torch = torch.sparse_coo_tensor(i, v, torch.Size(shape), device=device)

In [8]:
D2_torch.shape

torch.Size([109162, 11671655])

In [28]:
x_torch = torch.Tensor(x).to(device)

In [10]:
x_torch

tensor([0.9396, 0.9803, 0.2860,  ..., 0.3379, 0.9277, 0.5340], device='cuda:0')

In [11]:
D2_torch @ x_torch

tensor([237.8831, 222.3144, 218.6985,  ..., 232.4774, 219.7620, 217.2185],
       device='cuda:0')

In [12]:
timeit.timeit(lambda: D2_torch @ x_torch, number=5000)

151.07625597715378

In [13]:
D2_torch_coa = D2_torch.coalesce()

In [14]:
timeit.timeit(lambda: D2_torch_coa @ x_torch, number=5000)

4.34778428543359

In [29]:
D2_torch_csc = D2_torch_coa.to_sparse_csc()

In [16]:
timeit.timeit(lambda: D2_torch_csc @ x_torch, number=5000)

81.52595884166658

In [30]:
D2_torch_csr = D2_torch_coa.to_sparse_csr()

In [31]:
timeit.timeit(lambda: D2_torch_csr @ x_torch, number=5000)

1.8002560082823038

In [32]:
timeit.timeit(lambda: D2_torch_csr.mv(x_torch), number=5000)

1.9885549005120993

In [35]:
D2_torch_csr_T = D2_torch_coa.T.to_sparse_csr()

In [36]:
D2_torch_csr_T

tensor(crow_indices=tensor([       0,        3,        6,  ..., 35014959,
                            35014962, 35014965]),
       col_indices=tensor([     0,      1,    448,  ..., 109159, 109160,
                           109161]),
       values=tensor([ 1., -1.,  1.,  ...,  1., -1.,  1.]), device='cuda:0',
       size=(11671655, 109162), nnz=35014965, layout=torch.sparse_csr)

In [37]:
x_torch = x_torch.reshape((len(x_torch),1))

In [39]:
x_torch_T = x_torch.T
x_torch_T

tensor([[0.9396, 0.9803, 0.2860,  ..., 0.3379, 0.9277, 0.5340]],
       device='cuda:0')

In [40]:
x_torch_T.mm(D2_torch_csr_T)

tensor([[237.8831, 222.3144, 218.6985,  ..., 232.4774, 219.7620, 217.2185]],
       device='cuda:0')

In [41]:
timeit.timeit(lambda: x_torch_T.mm(D2_torch_csr_T), number=5000)

81.25843229424208

In [22]:
D2_torch_csr.mm(x_torch)

tensor([[237.8831],
        [222.3144],
        [218.6985],
        ...,
        [232.4774],
        [219.7620],
        [217.2185]], device='cuda:0')

In [61]:
timeit.timeit(lambda: D2_torch_coa_csr.mm(torch.rand((D2_torch.shape[1],1), device=device)), number=50000)

24.960342657985166

In [23]:
D2_torch_csr @ x_torch

tensor([[237.8831],
        [222.3144],
        [218.6985],
        ...,
        [232.4774],
        [219.7620],
        [217.2185]], device='cuda:0')